# Metadata extraction to SQL database

In our previous example, we have seen how to extract metadata from a lab notebook and store it in a data model. In this example, we will see how to stream data from an LLM directly into a SQL database.

For the sake of demonstration, this example will not make use of existing datasets, but rather let our LLM provide the data. Specifically, we will ask the LLM to provide the relevant molecules for several biochemical pathways.

However, the approach can easily be looped to existing datasets as well! Lets see how this works in practice.

In [1]:
from typing import Iterable

import rich
from instructor import OpenAISchema

from mdmodels import DataModel
from mdmodels.llm import query_openai
from mdmodels.sql import generate_sqlmodel, DatabaseConnector

In [3]:
# 1) Load the data model from the markdown file
dm = DataModel.from_markdown("model.md")

# 2) Generate the SQLModel classes
models = generate_sqlmodel(data_model=dm, base_classes=[OpenAISchema])

# 3) Create the database and tables
db = DatabaseConnector(database="")
db.create_tables(models)

In [6]:
# We need to open a session to interact with the database
with db as session:
    # Instruct the LLM to extract the molecules
    response = query_openai(
        response_model=Iterable[models.Molecule], # If we expect multiple molecules, we need to use Iterable
        query="Give me all molecules in the glycolysis, pentose phosphate and citric acid pathway.",
        pre_prompt="You are proficient in chemistry and biochemistry.",
        use_scaffold=False, # No need to provide thoughts here
    )

    session.add_all(response)

    # Commit the changes to the database
    session.commit()

    # Reset the session to clear the cache
    session.reset()

    # Verify that the data was correctly inserted
    results = session.exec(
        select(models.Molecule)
    ).all()

    rich.print(results)

[
    Molecule(name='Glucose', formula='C6H12O6', id=1),
    Molecule(name='Glucose-6-phosphate', formula='C6H13O9P', id=2),
    Molecule(name='Fructose-6-phosphate', formula='C6H13O9P', id=3),
    Molecule(name='Fructose-1,6-bisphosphate', formula='C6H14O12P2', id=4),
    Molecule(name='Glyceraldehyde-3-phosphate', formula='C3H7O6P', id=5),
    Molecule(name='Dihydroxyacetone phosphate', formula='C3H7O6P', id=6),
    Molecule(name='1,3-Bisphosphoglycerate', formula='C3H8O10P2', id=7),
    Molecule(name='3-Phosphoglycerate', formula='C3H7O7P', id=8),
    Molecule(name='2-Phosphoglycerate', formula='C3H7O7P', id=9),
    Molecule(name='Phosphoenolpyruvate', formula='C3H5O6P', id=10),
    Molecule(name='Pyruvate', formula='C3H4O3', id=11),
    Molecule(name='Ribulose-5-phosphate', formula='C5H11O8P', id=12),
    Molecule(name='Ribose-5-phosphate', formula='C5H11O8P', id=13),
    Molecule(name='Xylulose-5-phosphate', formula='C5H11O8P', id=14),
    Molecule(name='Sedoheptulose-7-phosphate', formula='C7H15O10P', id=15),
    Molecule(name='Erythrose-4-phosphate', formula='C4H9O7P', id=16),
    Molecule(name='6-Phosphogluconate', formula='C6H13O10P', id=17),
    Molecule(name='Citrate', formula='C6H8O7', id=18),
    Molecule(name='Isocitrate', formula='C6H8O7', id=19),
    Molecule(name='α-Ketoglutarate', formula='C5H6O5', id=20),
    Molecule(name='Succinyl-CoA', formula='C25H40N7O19P3S', id=21),
    Molecule(name='Succinate', formula='C4H6O4', id=22),
    Molecule(name='Fumarate', formula='C4H4O4', id=23),
    Molecule(name='Malate', formula='C4H6O5', id=24),
    Molecule(name='Oxaloacetate', formula='C4H4O5', id=25)
]

## Conclusion

In this example, we have seen how to stream data from an LLM directly into a SQL database. While this example is simple, the concept can be easily extended to more complex datasets and analysis pipelines. You can make use of all the tools you can learn from our [Database Example](./examples/sql/basic/SQLDatabaseExample.ipynb)!

Furthermore, the approach can easily be looped to existing datasets as well!